## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-19-01-11-01 +0000,startribune,"Maye throws 3 TD passes, Stroud has 4 INTs as ...",https://www.startribune.com/maye-throws-3-td-p...
1,2026-01-19-01-06-04 +0000,nypost,Snowy weather causes hundreds of cancellations...,https://nypost.com/2026/01/18/us-news/snowy-we...
2,2026-01-19-01-00-00 +0000,wsj,Europe Is Bracing for a New Trade Fight With T...,https://www.wsj.com/world/europe/europe-is-bra...
3,2026-01-19-01-00-00 +0000,wsj,Even MBAs From Top Business Schools Are Strugg...,https://www.wsj.com/lifestyle/careers/even-mba...
4,2026-01-19-00-53-04 +0000,bbc,"Factory explosion in China kills two, injures 84",https://www.bbc.com/news/articles/c5y2pg28lgjo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,44
142,greenland,20
123,ice,12
143,threats,7
154,one,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
173,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...,92
99,2026-01-18-18-20-27 +0000,nypost,EU threatens trade ‘bazooka’ against US after ...,https://nypost.com/2026/01/18/world-news/europ...,88
108,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...,88
122,2026-01-18-15-00-29 +0000,cbc,European nations warn of 'dangerous downward s...,https://www.cbc.ca/news/world/eight-countries-...,86
181,2026-01-18-06-01-39 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
173,92,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...
36,36,2026-01-18-23-24-23 +0000,nypost,Pregnant mother of two ID’d among six women ki...,https://nypost.com/2026/01/18/world-news/pregn...
40,34,2026-01-18-23-14-31 +0000,cbc,At least 20 believed dead in high-speed train ...,https://www.cbc.ca/news/world/spain-train-cras...
136,30,2026-01-18-13-00-00 +0000,startribune,Wondering what to do indoors with kids on a co...,https://www.startribune.com/best-indoor-playgr...
126,29,2026-01-18-14-56-16 +0000,nypost,President Trump puts $1B price tag on permanen...,https://nypost.com/2026/01/18/us-news/presiden...
106,27,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
93,27,2026-01-18-19-15-11 +0000,nypost,Stunned woman wakes up to find 8-foot python o...,https://nypost.com/2026/01/18/world-news/stunn...
108,24,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...
28,23,2026-01-18-23-56-26 +0000,nyt,European Union Officials Lean Toward Negotiati...,https://www.nytimes.com/2026/01/18/world/europ...
133,23,2026-01-18-13-25-03 +0000,bbc,Syrian army seizes country's largest oil field...,https://www.bbc.com/news/articles/cx20n55yxrlo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
